In [1]:
import numpy as np
import pandas as pd
# import sklearn

input_file = "kc_house_data.csv"
df = pd.read_csv(input_file)

In [2]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21613 entries, 735 to 15795
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long

In [4]:
corr_matrix=df.corr()

In [5]:
corr_matrix["price"].sort_values(ascending=False)

price            1.000000
sqft_living      0.702035
grade            0.667434
sqft_above       0.605567
sqft_living15    0.585379
bathrooms        0.525138
view             0.397293
sqft_basement    0.323816
bedrooms         0.308350
lat              0.307003
waterfront       0.266369
floors           0.256794
yr_renovated     0.126434
sqft_lot         0.089661
sqft_lot15       0.082447
yr_built         0.054012
condition        0.036362
long             0.021626
id              -0.016762
zipcode         -0.053203
Name: price, dtype: float64

In [6]:
df['total_rooms'] = df['sqft_living']+ df['bedrooms'] + df['bathrooms']
df['sqft_total_size'] = df['sqft_living'] + df['sqft_above'] + df['sqft_basement']
df['yard'] = df['sqft_lot']-df['sqft_living']
#df['b'] = df["sqft_living"] / df['sqft_living15']
df['d'] = df['grade'] * df['condition']
df['total_condition']=df['condition']/df['yr_built'] 
df["outsideview"] = df["waterfront"]+df["lat"]
df["location"]=df['long']*df['lat']
df["outsideview2"] = df["waterfront"]*df["view"]+df['lat']


In [7]:
housing_prepared = df.drop((["price","date","yr_renovated","bathrooms"]),axis=1)
housing_labels = df["price"].copy()


In [8]:
corr_matrix=df.corr()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21613 entries, 735 to 15795
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               21613 non-null  int64  
 1   date             21613 non-null  object 
 2   price            21613 non-null  float64
 3   bedrooms         21613 non-null  int64  
 4   bathrooms        21613 non-null  float64
 5   sqft_living      21613 non-null  int64  
 6   sqft_lot         21613 non-null  int64  
 7   floors           21613 non-null  float64
 8   waterfront       21613 non-null  int64  
 9   view             21613 non-null  int64  
 10  condition        21613 non-null  int64  
 11  grade            21613 non-null  int64  
 12  sqft_above       21613 non-null  int64  
 13  sqft_basement    21613 non-null  int64  
 14  yr_built         21613 non-null  int64  
 15  yr_renovated     21613 non-null  int64  
 16  zipcode          21613 non-null  int64  
 17  lat       

In [10]:
corr_matrix["price"].sort_values(ascending=False)

price              1.000000
sqft_living        0.702035
sqft_total_size    0.702035
total_rooms        0.701933
grade              0.667434
sqft_above         0.605567
sqft_living15      0.585379
bathrooms          0.525138
d                  0.490904
outsideview        0.404086
view               0.397293
outsideview2       0.366842
sqft_basement      0.323816
bedrooms           0.308350
lat                0.307003
waterfront         0.266369
floors             0.256794
yr_renovated       0.126434
sqft_lot           0.089661
sqft_lot15         0.082447
yard               0.074361
yr_built           0.054012
condition          0.036362
total_condition    0.032805
long               0.021626
id                -0.016762
zipcode           -0.053203
location          -0.265504
Name: price, dtype: float64

In [11]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'total_rooms',
       'sqft_total_size', 'yard', 'd', 'total_condition', 'outsideview',
       'location', 'outsideview2'],
      dtype='object')

In [12]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# ExtraTrees

In [13]:
from sklearn.ensemble import ExtraTreesRegressor

rnd_reg = ExtraTreesRegressor(n_estimators=1000, random_state=42,max_depth=30)
rnd_reg.fit(housing_prepared, housing_labels)

ExtraTreesRegressor(max_depth=30, n_estimators=1000, random_state=42)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesRegressor

rnd_reg = ExtraTreesRegressor(n_estimators=1000, random_state=42,max_depth=30)
rnd_reg.fit(housing_prepared, housing_labels)
temp = []
model = [rnd_reg]
def cv10(model, housing_prepared, housing_labels):
    model.fit(housing_prepared, housing_labels)
    model_scores = cross_val_score(model, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
    model_rmse_scores = np.sqrt(-model_scores)
    temp.append(model_rmse_scores)


In [15]:
for i in model:
    scores = cv10(i, housing_prepared,housing_labels)


In [16]:
pd.Series(*temp).mean()

123657.10572334626